In [1]:
from document_polluter import DocumentPolluter
import yaml
import os
import requests
import json
import statistics

with open('credentials.yaml') as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

In [2]:
with open('paragraphs/driving.yaml') as file:
    documents = yaml.load(file, Loader=yaml.FullLoader)

dp = DocumentPolluter(documents=documents, genre='gender')
len(dp.eligible_documents)

10

In [3]:
url = f"{credentials['azure']['endpoint']}/text/analytics/v2.1/sentiment"
headers = {'content-type': 'application/json', 'Ocp-Apim-Subscription-Key': credentials['azure']['key']}

sentiment = {}

for genre, documents in dp.polluted_documents.items():
    data = {'documents': []}
    
    for idx, document in enumerate(documents):
        data['documents'].append({"language": "en", "id": idx, "text": document})
        
    r = requests.post(url=url, data=json.dumps(data), headers=headers)
    
    results = json.loads(r.text)
    
    sentiment[genre] = [s['score'] for s in results['documents']]

In [4]:
print(f"""
Number of Samples: {len(dp.eligible_documents)}

Female tweet sentiment scores
Average: {sum(sentiment['female']) / len(sentiment['female'])}
Standard Deviation: {statistics.stdev(sentiment['female'])}

Male tweet sentiment scores
Average: {sum(sentiment['male']) / len(sentiment['male'])}
Standard Deviation: {statistics.stdev(sentiment['male'])}
""")


Number of Samples: 10

Female tweet sentiment scores
Average: 0.3520379900932312
Standard Deviation: 0.2928391766712977

Male tweet sentiment scores
Average: 0.3520379900932312
Standard Deviation: 0.2928391766712977



In [5]:
results = []
for idx, document in enumerate(dp.eligible_documents):
    results.append({
        'female_sentence': dp.polluted_documents['female'][idx],
        'male_sentence': dp.polluted_documents['male'][idx],
        'female_score': sentiment['female'][idx],
        'male_score': sentiment['male'][idx],
        'difference': abs(sentiment['female'][idx] - sentiment['male'][idx])
    })

In [6]:
list(filter(lambda x: x['difference'] != 0, results))


[]